In [1]:
import os
import pathlib
import numpy as np
import click
import json
import torch

#from rlkit.envs import ENVS
from rlkit.envs.wrappers import NormalizedBoxEnv
from rlkit.torch.sac.policies import TanhGaussianPolicy
from rlkit.torch.networks import FlattenMlp, MlpEncoder, RecurrentEncoder
from rlkit.torch.sac.sac import PEARLSoftActorCritic
from rlkit.torch.sac.agent import PEARLAgent
from rlkit.launchers.launcher_util import setup_logger
import rlkit.torch.pytorch_util as ptu
from configs.default import default_config

In [2]:
# default PEARL experiment settings
# all experiments should modify these settings only as needed
default_config = dict(
    env_name='hjgzuighi',
    n_train_tasks=2,
    n_eval_tasks=2,
    latent_size=2, # dimension of the latent context vector
    net_size=10, # number of units per FC layer in each network
    path_to_weights=None, # path to pre-trained weights to load into networks
    env_params=dict(
        n_tasks=2, # number of distinct tasks in this domain, shoudl equal sum of train and eval tasks
        randomize_tasks=True, # shuffle the tasks after creating them
    ),
    algo_params=dict(
        meta_batch=16, # number of tasks to average the gradient across
        num_iterations=500, # number of data sampling / training iterates
        num_initial_steps=2000, # number of transitions collected per task before training
        num_tasks_sample=5, # number of randomly sampled tasks to collect data for each iteration
        num_steps_prior=400, # number of transitions to collect per task with z ~ prior
        num_steps_posterior=0, # number of transitions to collect per task with z ~ posterior
        num_extra_rl_steps_posterior=400, # number of additional transitions to collect per task with z ~ posterior that are only used to train the policy and NOT the encoder
        num_train_steps_per_itr=2000, # number of meta-gradient steps taken per iteration
        num_evals=2, # number of independent evals
        num_steps_per_eval=600,  # nuumber of transitions to eval on
        batch_size=256, # number of transitions in the RL batch
        embedding_batch_size=64, # number of transitions in the context batch
        embedding_mini_batch_size=64, # number of context transitions to backprop through (should equal the arg above except in the recurrent encoder case)
        max_path_length=200, # max path length for this environment
        discount=0.99, # RL discount factor
        soft_target_tau=0.005, # for SAC target network update
        policy_lr=3E-4,
        qf_lr=3E-4,
        vf_lr=3E-4,
        context_lr=3e-4,
        reward_scale=5., # scale rewards before constructing Bellman update, effectively controls weight on the entropy of the policy
        sparse_rewards=False, # whether to sparsify rewards as determined in env
        kl_lambda=.1, # weight on KL divergence term in encoder loss
        use_information_bottleneck=True, # False makes latent context deterministic
        use_next_obs_in_context=False, # use next obs if it is useful in distinguishing tasks
        update_post_train=1, # how often to resample the context when collecting data during training (in trajectories)
        num_exp_traj_eval=1, # how many exploration trajs to collect before beginning posterior sampling at test time
        recurrent=False, # recurrent or permutation-invariant encoder
        dump_eval_paths=False, # whether to save evaluation trajectories
    ),
    util_params=dict(
        base_log_dir='output',
        use_gpu=False,
        gpu_id=0,
        debug=False, # debugging triggers printing and writes logs to debug directory
        docker=False, # TODO docker is not yet supported
    )
)


In [3]:
class space():
    flat_dim = 3
    high = 0.1
    low = -0.1
    pass

In [4]:
class task():
    
    def __init__(self):
        #self.observation_space = {"low":-np.inf, "high":np.inf,'flat_dim':2}
        self.observation_space = space()
        print(self.observation_space.flat_dim)
        
        self.action_space = space()
        #self._goal = [-1,1,0] #needed ? correct?
    def reset_task(self, idx):
        pass
    
    def reset(self):
        return np.asarray([0,1,2])
    
    def step(self, a):
        print('ACTIOIN:', a)
        
        
        forward_reward = 10-np.sum(np.abs(a-[-1,1,0]))
        ctrl_cost = 0. # .5 * np.square(a).sum()
        contact_cost = 0.
        survive_reward = 0. # 1.0
        
        
        done = False #will it ever terminate tho?
        reward = forward_reward - ctrl_cost - contact_cost + survive_reward
        print('reward:', reward)
        return np.asarray([0,1,2]), reward, done, dict(
            reward_forward=forward_reward,
            reward_ctrl=-ctrl_cost,
            reward_contact=-contact_cost,
            reward_survive=survive_reward,
            torso_velocity=forward_reward,
        )

In [5]:

def experiment(variant):

    # create multi-task environment and sample tasks
    
    TAKS = task()
    
    env = TAKS#'yolo'#NormalizedBoxEnv(ENVS[variant['env_name']](**variant['env_params']))
    tasks = [0,1,2]
    obs_dim = int(3)
    action_dim = int(3)
    reward_dim = 1

    # instantiate networks
    latent_dim = variant['latent_size']
    context_encoder_input_dim = 2 * obs_dim + action_dim + reward_dim if variant['algo_params']['use_next_obs_in_context'] else obs_dim + action_dim + reward_dim
    context_encoder_output_dim = latent_dim * 2 if variant['algo_params']['use_information_bottleneck'] else latent_dim
    net_size = variant['net_size']
    recurrent = variant['algo_params']['recurrent']
    encoder_model = RecurrentEncoder if recurrent else MlpEncoder

    context_encoder = encoder_model(
        hidden_sizes=[200, 200, 200],
        input_size=context_encoder_input_dim,
        output_size=context_encoder_output_dim,
    )
    qf1 = FlattenMlp(
        hidden_sizes=[net_size, net_size, net_size],
        input_size=obs_dim + action_dim + latent_dim,
        output_size=1,
    )
    qf2 = FlattenMlp(
        hidden_sizes=[net_size, net_size, net_size],
        input_size=obs_dim + action_dim + latent_dim,
        output_size=1,
    )
    vf = FlattenMlp(
        hidden_sizes=[net_size, net_size, net_size],
        input_size=obs_dim + latent_dim,
        output_size=1,
    )
    policy = TanhGaussianPolicy(
        hidden_sizes=[net_size, net_size, net_size],
        obs_dim=obs_dim + latent_dim,
        latent_dim=latent_dim,
        action_dim=action_dim,
    )
    agent = PEARLAgent(
        latent_dim,
        context_encoder,
        policy,
        **variant['algo_params']
    )
    algorithm = PEARLSoftActorCritic(
        env=env,
        train_tasks=list(tasks[:variant['n_train_tasks']]),
        eval_tasks=list(tasks[-variant['n_eval_tasks']:]),
        nets=[agent, qf1, qf2, vf],
        latent_dim=latent_dim,
        **variant['algo_params']
    )
    print('herefutff')
    # optionally load pre-trained weights
    if variant['path_to_weights'] is not None:
        path = variant['path_to_weights']
        context_encoder.load_state_dict(torch.load(os.path.join(path, 'context_encoder.pth')))
        qf1.load_state_dict(torch.load(os.path.join(path, 'qf1.pth')))
        qf2.load_state_dict(torch.load(os.path.join(path, 'qf2.pth')))
        vf.load_state_dict(torch.load(os.path.join(path, 'vf.pth')))
        # TODO hacky, revisit after model refactor
        algorithm.networks[-2].load_state_dict(torch.load(os.path.join(path, 'target_vf.pth')))
        policy.load_state_dict(torch.load(os.path.join(path, 'policy.pth')))
    print('zhtgdtseru')
    # optional GPU mode
    #ptu.set_gpu_mode(variant['util_params']['use_gpu'], variant['util_params']['gpu_id'])
    #if ptu.gpu_enabled():
        #algorithm.to()

    # debugging triggers a lot of printing and logs to a debug directory
    DEBUG = variant['util_params']['debug']
    os.environ['DEBUG'] = str(int(DEBUG))
    print('löllll')

    # create logging directory
    # TODO support Docker
##############    
    
    exp_id = 'debug' if DEBUG else None
    experiment_log_dir = setup_logger(variant['env_name'], variant=variant, exp_id=exp_id, base_log_dir=variant['util_params']['base_log_dir'])
    
##############    
    
    print('hgdftzrd')
    # optionally save eval trajectories as pkl files
    if variant['algo_params']['dump_eval_paths']:
        pickle_dir = experiment_log_dir + '/eval_trajectories'
        pathlib.Path(pickle_dir).mkdir(parents=True, exist_ok=True)

    # run the algorithm
    print('daa')
    algorithm.train()

def deep_update_dict(fr, to):
    ''' update dict of dicts with new values '''
    # assume dicts have same keys
    for k, v in fr.items():
        if type(v) is dict:
            deep_update_dict(v, to[k])
        else:
            to[k] = v
    return to

def main():

    variant = default_config

    experiment(variant)

main()

3
herefutff
zhtgdtseru
löllll
2020-12-01 23:46:50.526266 CET | Variant:
2020-12-01 23:46:50.526836 CET | {
  "env_name": "hjgzuighi",
  "n_train_tasks": 2,
  "n_eval_tasks": 2,
  "latent_size": 2,
  "net_size": 10,
  "path_to_weights": null,
  "env_params": {
    "n_tasks": 2,
    "randomize_tasks": true
  },
  "algo_params": {
    "meta_batch": 16,
    "num_iterations": 500,
    "num_initial_steps": 2000,
    "num_tasks_sample": 5,
    "num_steps_prior": 400,
    "num_steps_posterior": 0,
    "num_extra_rl_steps_posterior": 400,
    "num_train_steps_per_itr": 2000,
    "num_evals": 2,
    "num_steps_per_eval": 600,
    "batch_size": 256,
    "embedding_batch_size": 64,
    "embedding_mini_batch_size": 64,
    "max_path_length": 200,
    "discount": 0.99,
    "soft_target_tau": 0.005,
    "policy_lr": 0.0003,
    "qf_lr": 0.0003,
    "vf_lr": 0.0003,
    "context_lr": 0.0003,
    "reward_scale": 5.0,
    "sparse_rewards": false,
    "kl_lambda": 0.1,
    "use_information_bottleneck": t

ACTIOIN: [-0.37542653 -0.7710982   0.16903244]
reward: 7.435295894742012
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.37245     0.30779788 -0.11989341]
reward: 8.560354463756084
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.63687354 -0.8395117   0.73969   ]
reward: 5.783924758434296
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.7719075 -0.3680306 -0.6707869]
reward: 6.189274966716766
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.93309313  0.32549223  0.18952179]
reward: 9.069063574075699
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.03292174 -0.16810371 -0.27402246]
reward: 7.590795565396547
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.2031387  -0.46474987 -0.03919432]
reward: 7.699194498360157
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.5011103  -0.89874685  0.367163  ]
reward: 6.232979834079742
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [0.13749234 0.92557335 0.40180907]
reward: 8.386271938681602
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.846207    0.58254915  0.985835

ACTIOIN: [-0.42107937  0.42436704 -0.65512836]
reward: 8.190318048000336
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.9737618 -0.5093582 -0.6931495]
reward: 5.82373046875
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.7899817  -0.48869136  0.6059908 ]
reward: 7.695299535989761
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.83704674 -0.7650888   0.30426762]
reward: 7.76769033074379
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [0.3572793  0.79547656 0.42917067]
reward: 8.00902658700943
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.2932716  -0.96558636  0.7595838 ]
reward: 5.981558263301849
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.6373267  -0.3117684   0.90818524]
reward: 6.142719626426697
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.69495314 -0.57541895 -0.6184236 ]
reward: 7.5011106133461
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.11806749 -0.5006669  -0.83802164]
reward: 6.7793789356946945
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.23234025 -0.1362147  -0.41788206]
rew

ACTIOIN: [-0.4444314 -0.4039128  0.0145671]
reward: 8.025951483286917
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.48790303  0.9171947  -0.9701504 ]
reward: 7.459141284227371
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.13315327 -0.48371983 -0.36990446]
reward: 7.279528990387917
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.24006276 -0.49555695  0.70201623]
reward: 7.042489573359489
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8715101  0.7712866  0.8964784]
reward: 8.746318280696869
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.08701189 -0.5395558   0.97655505]
reward: 6.570901051163673
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.77097094  0.91139746  0.9755895 ]
reward: 8.706778883934021
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.5813843  -0.9833389  -0.84602576]
reward: 5.5892510414123535
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.1345705   0.69338834  0.94509685]
reward: 7.882861986756325
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.7603047   0.26388553  0.90241

ACTIOIN: [-0.11970051 -0.13322362 -0.12971604]
reward: 7.85676085203886
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.15342116 -0.09942313  0.79689634]
reward: 6.950259372591972
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.7366942   0.82946205  0.49575722]
reward: 9.070399045944214
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [0.045881   0.342699   0.00180525]
reward: 8.295012741233222
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.1003516  -0.50443906  0.9657478 ]
reward: 6.429461568593979
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.24700226  0.07334693 -0.9775148 ]
reward: 7.342834383249283
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.17439798 -0.13257982  0.75436133]
reward: 6.938660874962807
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.16508579 -0.49424508 -0.89095104]
reward: 6.779889672994614
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [0.68959063 0.9802755  0.17749257]
reward: 8.113192304968834
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [ 0.6619932  -0.31847134  0.0806919

...

ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.

ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.00208123]
reward: 8.978175207274035
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.4923646   0.48789185 -0.

ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.00244072]
reward: 9.759090868523344
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8816572   0.8798744   0.

ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.00239298]
reward: 9.755790499504656
Observation: [0 1 2]
obs: [0 1 2]
ACTIOIN: [-0.8799905   0.87819296  0.

AttributeError: 'task' object has no attribute '_goal'